In [1]:
import xarray 
import os
import netCDF4
import numpy as np
import matplotlib.pyplot as plt
import imdlib as imd
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import scipy.io
import matplotlib.image as mpimg
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

In [5]:
x1 =pd.read_csv(r"C:\Users\ratho\Desktop\Software Group Project - III\Climate Data Modelling\Our Work\CSV Excel Data\SSTA Category 1901-2010.csv",index_col=0)
y1 =pd.read_csv(r"C:\Users\ratho\Desktop\Software Group Project - III\Climate Data Modelling\Our Work\Codes\Main Codes\SST Clusters\SSTA 8 Cluster Number 1901-2010.csv",index_col=0)
x2 =pd.read_csv(r"C:\Users\ratho\Desktop\Software Group Project - III\Climate Data Modelling\Our Work\CSV Excel Data\SSTA Category 2011-2022.csv",index_col=0)
y2 =pd.read_csv(r"C:\Users\ratho\Desktop\Software Group Project - III\Climate Data Modelling\Our Work\Codes\Main Codes\SST Clusters\SSTA 8 Cluster Number 2011-2022.csv",index_col=0)

In [6]:
x=pd.concat([x1,x2])
x=x.set_index(i for i in range(122))
y=pd.concat([y1,y2])
y=y.set_index(i for i in range(122))


In [7]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.1,random_state=42)

In [8]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LinearRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis


In [9]:
model_params = {
    'Support Vector Machine': {
        'Model': svm.SVC(random_state=42),
        'Params': {
            'C': [0.1,1, 10, 20, 50, 100],
            'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
            'gamma' : ['scale','auto'],
            'degree': [2, 3, 4],
            'coef0': [0.0, 0.1, 0.5,1.0],
            'probability' : [True,False]
        }
    },
    'Random Forest': {
        'Model': RandomForestClassifier(random_state=42),
        'Params': {
            'n_estimators': [1, 5, 10, 20, 50],
            'criterion': ['gini', 'entropy'],
            'max_depth': [None, 10, 20, 30],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4],
        }
    },
    'Logistic Regression': {
        'Model': LogisticRegression(multi_class='ovr',random_state=42),
        'Params': {
            'C': [0.1,1, 10, 20, 50, 100],
            'penalty': ['l1', 'l2'],
            'solver': ['liblinear']
        }
    },
    'Naive Bayes Gaussian': {
        'Model': GaussianNB(),
        'Params': {}
    },
    'Naive Bayes Multinomial': {
        'Model': MultinomialNB(),
        'Params': {
            'alpha': [0.1, 0.5, 1.0, 2.0],
        }
    },
    'Decision Tree': {
        'Model': DecisionTreeClassifier(random_state=42),
        'Params': {
            'criterion': ['gini', 'entropy'],
            'max_depth': [None, 10, 20, 30],
            'min_samples_split': [None,1, 2, 5, 10],
            'min_samples_leaf': [None,1, 2, 4, 8, 16],
            'max_features': ['sqrt', 'log2', None]
        }
    },
    'Linear Discriminant Analysis': {
        'Model': LinearDiscriminantAnalysis(),
        'Params' : {

        }
    },
}



In [10]:
y_train_array=y_train.values.ravel()
y_train_array

array([2, 2, 3, 1, 1, 2, 3, 0, 4, 5, 4, 0, 0, 2, 6, 4, 2, 7, 0, 2, 3, 2,
       6, 4, 1, 1, 1, 7, 0, 5, 0, 7, 2, 2, 3, 3, 6, 1, 0, 3, 7, 0, 0, 4,
       7, 5, 0, 6, 1, 3, 5, 2, 2, 7, 3, 5, 7, 7, 3, 2, 0, 6, 2, 6, 0, 3,
       6, 5, 2, 5, 2, 6, 2, 1, 1, 0, 4, 2, 1, 0, 0, 2, 1, 1, 2, 0, 3, 2,
       3, 1, 3, 3, 3, 2, 4, 2, 2, 0, 6, 6, 2, 0, 4, 5, 2, 1, 6, 4, 2],
      dtype=int64)

In [11]:
from sklearn.model_selection import GridSearchCV
import pandas as pd
scores = []

for model_name, mp in model_params.items():
    clf =  GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
    clf.fit(x_train, y_train_array)
    y_pred = clf.predict(x_test)
    y_pred = np.ceil(y_pred)
    y_test_array= y_test.values.ravel()
    y_test_array
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'accuracy_score': accuracy_score(y_pred,y_test_array),
        'best_params': clf.best_params_,
    })
    
df = pd.DataFrame(scores,columns=['model','best_score','accuracy_score','best_params'])
df

,model,best_score,accuracy_score,best_params
0,svm,0.926840,1.000000,"{'C': 10, 'coef0': 0.0, 'degree': 2, 'kernel':..."
1,random_forest,0.789177,1.000000,"{'criterion': 'gini', 'max_depth': None, 'min_..."
2,logistic_regression,0.733333,0.692308,"{'C': 10, 'penalty': 'l2'}"
3,naive_bayes_gaussian,0.816017,0.846154,{}
4,naive_bayes_multinomial,0.595671,0.538462,{'alpha': 0.1}
5,decision_tree,0.669697,0.692308,"{'criterion': 'gini', 'max_depth': None, 'min_..."
6,linear_discriminant_analysis,0.870996,0.923077,{}


In [12]:
df.to_csv(r"C:\Users\ratho\Desktop\Software Group Project - III\Climate Data Modelling\Our Work\Codes\Main Codes\Results\SSTA Clusters Train Test Split Results.csv")